In [41]:
import pandas as pd

In [42]:
dfs = []
for name in os.listdir('./player_game_logs'):
    dfs.append(pd.read_csv(f"./player_game_logs/{name}/{name}_SUMMARY.csv"))
combined_df = pd.concat(dfs, ignore_index=True).set_index('model_type')

In [43]:
summary_df = combined_df.groupby(combined_df.index).mean()
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_columns', None)
summary_df

,n_train,n_test,mse_train,rmse_train,r2_train,mse_test,rmse_test,r2_test,y_test_mean,bias,Game of season_weight,Game for player_weight,Age (days)_weight,Home?_weight,Game Started?_weight,Year_weight,Month of Season_weight,Team_GSW_weight,Opponent_GSW_weight,Team_DET_weight,Opponent_DET_weight,Team_WSB_weight,Opponent_WSB_weight,Team_LAC_weight,Opponent_LAC_weight,Team_DAL_weight,Opponent_DAL_weight,Team_BOS_weight,Opponent_BOS_weight,Team_MEM_weight,Opponent_MEM_weight,Team_UTA_weight,Opponent_UTA_weight,Team_MIN_weight,Opponent_MIN_weight,Team_NJN_weight,Opponent_NJN_weight,Team_TOR_weight,Opponent_TOR_weight,Team_BRK_weight,Opponent_BRK_weight,Team_DEN_weight,Opponent_DEN_weight,Team_WAS_weight,Opponent_WAS_weight,Team_PHI_weight,Opponent_PHI_weight,Team_POR_weight,Opponent_POR_weight,Team_IND_weight,Opponent_IND_weight,Team_MIL_weight,Opponent_MIL_weight,Team_SAC_weight,Opponent_SAC_weight,Team_ATL_weight,Opponent_ATL_weight,Team_ORL_weight,Opponent_ORL_weight,Team_VAN_weight,Opponent_VAN_weight,Team_OKC_weight,Opponent_OKC_weight,Team_LAL_weight,Opponent_LAL_weight,Team_SDC_weight,Opponent_SDC_weight,Team_NOH_weight,Opponent_NOH_weight,Team_NYK_weight,Opponent_NYK_weight,Team_CHH_weight,Opponent_CHH_weight,Team_SEA_weight,Opponent_SEA_weight,Team_NOP_weight,Opponent_NOP_weight,Team_NOK_weight,Opponent_NOK_weight,Team_PHO_weight,Opponent_PHO_weight,Team_CHA_weight,Opponent_CHA_weight,Team_MIA_weight,Opponent_MIA_weight,Team_CLE_weight,Opponent_CLE_weight,Team_CHO_weight,Opponent_CHO_weight,Team_CHI_weight,Opponent_CHI_weight,Team_SAS_weight,Opponent_SAS_weight,Team_KCK_weight,Opponent_KCK_weight,Team_HOU_weight,Opponent_HOU_weight
model_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Lasso,402.0171,101.0063,34.6428,5.7129,0.0477,35.1239,5.7259,0.0265,8.7449,8.3199,0.0000,0.0004,0.0409,0.0003,0.6546,0.0005,0.0000,0.0094,0.0000,0.0061,0.0000,0.0057,0.0000,0.0055,0.0000,0.0043,0.0000,0.0051,0.0000,-0.0040,0.0000,0.0049,0.0000,0.0048,0.0000,0.0085,0.0000,0.0046,0.0000,-0.0002,0.0000,0.0108,0.0000,0.0019,0.0000,0.0091,0.0000,0.0046,0.0000,0.0093,0.0000,0.0040,0.0000,0.0018,0.0000,0.0085,0.0000,0.0109,0.0000,0.0017,0.0000,-0.0029,0.0000,-0.0050,0.0000,0.0000,0.0000,0.0027,0.0000,0.0018,0.0000,0.0018,0.0000,0.0077,0.0000,0.0001,0.0000,0.0000,0.0000,0.0031,0.0000,0.0036,0.0000,0.0020,0.0000,-0.0003,0.0000,0.0033,0.0000,0.0015,0.0000,-0.0030,0.0000,0.0009,0.0000,0.0069,0.0000
LinReg,402.0171,101.0063,24.6829,4.7838,0.3372,"967,609,371,024,295,842,545,664.0000","109,882,059,977.8117","-28,802,860,475,967,061,622,784.0000",8.7449,"-148,976,774,418.1524","2,063,672,095,980.2119","-2,063,672,095,978.3381",0.1518,0.2561,"39,723,741,719.2936",-0.5979,-0.4375,"202,730,332,944.3262","165,212,290,079.8911","655,898,280,080.2975","112,099,261,004.2770","94,448,208,592.2380","-583,256,900,660.9479","250,633,413,064.3935","74,180,214,190.6227","234,840,666,402.5617","113,179,915,613.1286","-217,610,382,724.7328","86,363,639,452.6838","-103,229,946,563.7150","134,189,267,402.8597","-29,625,315,781.1379","83,270,275,106.9926","186,819,066,543.3286","247,791,335,614.4087","194,670,665,950.8740","72,539,308,905.3854","817,609,215,546.6450","154,300,069,066.2872","67,643,174,462.1048","25,901,800,055.4824","351,980,973,199.0768","153,818,705,368.9506","-46,283,336,652.2670","211,092,793,504.6075","-185,105,221,632.4011","82,234,964,476.5919","329,243,838,186.7130","131,557,898,497.0161","120,815,144,249.0456","135,802,040,207.2835","-99,321,780,787.7578","114,815,261,906.0329","265,872,510,847.4184","107,715,602,970.4321","236,943,529,912.1367","27,826,968,548.0326","307,953,364,069.8708","114,613,831,862.9159","-300,569,175,257.7212","40,510,300,496.7839","-404,002,933,914.0988","-40,837,903,324.0544","-417,439,794,463.2264","87,854,675,942.6641","52,945,934,323.0535","-25,816,977,161.4017","154,732,035,522.5981","114,933,523,503.1067","-161,885,085,899.5002","127,304,533,743.6776","144,796,855,